# YouTube Video Recommender System

Library Installations and Imports

In [3]:
# Get updates on what's going on in the background
!pip install typing-inspect==0.8.0 \
typing_extensions==4.5.0 \
-U FlagEmbedding \
protobuf==3.20.* \
peft \
-q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 6.1 MB/s eta 0:00:00


In [15]:
import pandas as pd
import numpy as np
from scipy import spatial
import json
from FlagEmbedding import FlagModel

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading, Understanding, and Cleaning the Data

In [7]:
df = pd.read_csv('/content/GB_videos_data.csv')

# Dropping the duplicated rows based on title
df = df.drop_duplicates(subset='title')
len(df)

1370

In [8]:
# Looking at the top 5 rows of the data frame
df.head()

,Unnamed: 0,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,0,aHzKJOC19SU,Milan 0-3 Inter | Inter Go Four Points Clear w...,2021-02-21T17:07:45Z,UCBJeMCIeLQos7wacox4hmLQ,Serie A,17,21.22.02,Ronaldo|Serie A|Dybala|highlights|Juventus|AC ...,3074832,64812,1759,4797,https://i.ytimg.com/vi/aHzKJOC19SU/default.jpg,False,False,Inter breezed past Milan in the derby to go fo...
1,1,-yI7PU5Sxj0,PE With Joe 2021 | Monday 22nd Feb,2021-02-22T09:29:11Z,UCAxW1XT0iEJo0TYlRfn6rYQ,The Body Coach TV,17,21.22.02,the body coach|Joe wicks|joe wix|the bodycoach...,92847,4809,140,362,https://i.ytimg.com/vi/-yI7PU5Sxj0/default.jpg,False,False,
2,2,vMcB2TcqD-A,Cards Against Humanity PHAN EDITION!,2021-02-21T20:29:11Z,UCHUE4ypXKp7ZkmdWbGJNgJg,AmazingPhil,22,21.22.02,dan and phil|dan howell|phil lester|funny|card...,511276,110828,114,10328,https://i.ytimg.com/vi/vMcB2TcqD-A/default.jpg,False,False,We play an offensive card game ..created by Da...
3,3,z8eMtJJFSJE,HIGHLIGHTS! David Avanesyan crushes Josh Kelly...,2021-02-21T07:00:30Z,UC_JQGBtA7P0RwkRxd7xpJcA,Sky Sports Boxing,17,21.22.02,sky sports boxing|sky sports boxing stream|box...,266111,2536,115,1454,https://i.ytimg.com/vi/z8eMtJJFSJE/default.jpg,False,False,►SUBSCRIBE http://bit.ly/SSBoxingSubJosh Kelly...
4,4,PYLqcf6SelQ,WHO IS THE BEST COUPLE?!,2021-02-21T06:11:13Z,UCPpATKqmMV-CNRNWYaDUwiA,Alexa Rivera,26,21.22.02,[none],2768920,183604,2567,12930,https://i.ytimg.com/vi/PYLqcf6SelQ/default.jpg,False,False,"This was so crazy, you will never guess who ma..."


In [9]:
# Getting a sample title - first row
df.iloc[0]['title']

'Milan 0-3 Inter | Inter Go Four Points Clear with HUGE Derby Win! | Serie A TIM'

In [10]:
# Getting a sample description - first row
print(df.iloc[0]['description'])

Inter breezed past Milan in the derby to go four points clear at the top of the table thanks to Romelu Lukaku and Lautaro Martinez! | Serie A TIMThis is the official channel for the Serie A, providing all the latest highlights, interviews, news and features to keep you up to date with all things Italian football.Subscribe to the channel here! https://bit.ly/2OM2Eax Find out more about the Serie A at: http://www.legaseriea.it/en/ Questo è il canale ufficiale della Serie A, dove potrai avere accesso ai momenti salienti, alle interviste, alle notizie e alle funzionalità del momento per rimanere aggiornato sulle ultime novità del campionato.Iscriviti qui al canale! https://bit.ly/2OM2EaxPer maggiori informazioni sulla Serie A: http://www.legaseriea.it/it  


In [11]:
# Number of unique titles
df['title'].nunique()

1370

In [12]:
# Keeping only the neccesary columns
df = df[['title', 'description', 'view_count', 'likes']]

In [13]:
# Lowercasing the titles and saving them into a new column
df['cleaned_title'] = df['title'].str.lower()


In [14]:
# Getting 5 random samples from the data frame
df.sample(5)

,title,description,view_count,likes,cleaned_title
519,This Channel Will Be 15 Years Old Tomorrow,https://ashensmovie.comThere's about 18 hours ...,72453,10005,this channel will be 15 years old tomorrow
833,Jeremy Clarkson reveals the car he regrets sel...,You may remember our Clarkson Q&A and F1 rant ...,499355,41240,jeremy clarkson reveals the car he regrets sel...
4045,I FOUND RARE BEANIE BABIES IN MY MOM’S GARAGE!,I went through my entire beanie baby collectio...,554139,28445,i found rare beanie babies in my mom’s garage!
1608,WTF WAS THIS PICK?! 😱 20 x PRIME OR MID ICON P...,twitter: https://www.twitter.com/runthefutmark...,216049,8459,wtf was this pick?! 😱 20 x prime or mid icon p...
2634,VFX Artists React to Bad & Great CGi 40,Sponsored by Audible. Visit https://audible.co...,975099,56589,vfx artists react to bad & great cgi 40


# Generating Word Embeddings

In [20]:
embedding_model = FlagModel('BAAI/bge-large-en-v1.5',
                    use_fp16=True)

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [18]:
def embed(text):
  return embedding_model.encode(text).astype(np.float32)

In [21]:
embed('Hello')

array([ 0.05105215,  0.00719567,  0.00387395, ..., -0.03147079,
       -0.03555793, -0.01076886], dtype=float32)

In [22]:
len(embed('Hello'))

1024

In [30]:
embedding_list = []

for title in df['cleaned_title']:
    embedding_list.append(embed(title))

print('Generating embeddings finished.')


Generating embeddings finished.


In [31]:
# Looking at one sample of the word embeddings
embedding_list[0]

array([ 1.9154741e-03,  3.5581475e-03, -2.9562460e-02, ...,
       -2.9091120e-03, -6.3435145e-05, -8.2318261e-03], dtype=float32)

In [32]:
# Saving the embeddings as a new column into the data frame
df['title_embeddings'] = embedding_list

In [ ]:
# It would be best to save the dataframe with the embeddings into a csv file,
# This way we do not have to generate embeddings each time, we just load the csv file

df.to_csv('/content/drive/MyDrive/GB_videos_data_embeddings.csv')

# df = df.read_csv('embeddings_GB_videos_data.csv')

In [33]:
df.head()

,title,description,view_count,likes,cleaned_title,title_embeddings
0,Milan 0-3 Inter | Inter Go Four Points Clear w...,Inter breezed past Milan in the derby to go fo...,3074832,64812,milan 0-3 inter | inter go four points clear w...,"[0.0019154741, 0.0035581475, -0.02956246, -0.0..."
1,PE With Joe 2021 | Monday 22nd Feb,,92847,4809,pe with joe 2021 | monday 22nd feb,"[0.03231064, 0.027919704, -0.03166256, 0.05026..."
2,Cards Against Humanity PHAN EDITION!,We play an offensive card game ..created by Da...,511276,110828,cards against humanity phan edition!,"[0.0037798083, 0.028637487, 0.028344473, 0.033..."
3,HIGHLIGHTS! David Avanesyan crushes Josh Kelly...,►SUBSCRIBE http://bit.ly/SSBoxingSubJosh Kelly...,266111,2536,highlights! david avanesyan crushes josh kelly...,"[0.013086135, 0.0016118499, -0.008401231, 0.02..."
4,WHO IS THE BEST COUPLE?!,"This was so crazy, you will never guess who ma...",2768920,183604,who is the best couple?!,"[0.051723946, 0.006681839, -0.044657722, 0.071..."


# Using the Embedding and Similarity Score to Recommend Videos

In [34]:
query = "funny videos of pets"

# Encoding the query
query_encoding = embed(query)

In [35]:
# Checking the word embeddings of the query
query_encoding

array([0.00373282, 0.013482  , 0.01556434, ..., 0.00813133, 0.01972056,
       0.01346161], dtype=float32)

In [36]:
# Finding the closest titles to the query by using cosine similarity.

df['similarity_score'] = df['title_embeddings'].apply(lambda x: 1-spatial.distance.cosine(x, query_encoding))
df_results = df.sort_values(by=['similarity_score'], ascending=False)

In [37]:
# The cosine similarity score have been added as a new column and have been ranked
df_results.head()

,title,description,view_count,likes,cleaned_title,title_embeddings,similarity_score
4212,I played weird virtual pet games,video game animals♥ The Team ♥Atrox: https://t...,2035404,253119,i played weird virtual pet games,"[0.0132928295, -0.0096675195, 0.007726326, 0.0...",0.709317
3822,What is this cute puppy dog doing?#Shorts,What is this cute puppy dog doing?#Shorts,656500,0,what is this cute puppy dog doing?#shorts,"[0.0075999214, 0.037784304, 0.0071791727, 0.00...",0.692028
4650,Invisible rope prank / TwinsFromRussia tiktok ...,Follow us on Instagram: @TwinsFromRussia #shorts,2161571,117298,invisible rope prank / twinsfromrussia tiktok ...,"[0.0025464355, 0.031353272, -0.019823503, 0.04...",0.636005
5802,I Made This Video 2 Years Ago,I filmed this video 2 years agoFollow my Twitt...,1495143,238784,i made this video 2 years ago,"[-0.023677675, 0.043644074, 0.0037068618, 0.04...",0.628972
802,PewDiePie Comment On This Video,please tell pewds to comment on this! #shorts ...,2833532,278436,pewdiepie comment on this video,"[0.010122138, 0.03768475, -0.012471712, 0.0304...",0.622203


# Wrapping the Recommending Steps in a Function

In [38]:
# Wrapping the steps above into a function

def recommend(query, k_recommendations):
  # This function gets two inputs, one is the query, the other one is the number of recommendations

  # Generating the word embeddings for the query
  query_encoding = embed(query)

  # Calculating the cosine similaties between the query embeddings and all the title embeddings
  df['similarity_score'] = df['title_embeddings'].apply(lambda x: 1-spatial.distance.cosine(x, query_encoding))
  # Ranking the data based on the cosine similarity scores
  df_results = df.sort_values(by=['similarity_score'], ascending=False)
  # Getting the top k recommendations
  recommendations = df_results['title'][:k_recommendations]
  # Printing all the top K recommendations
  for recom in recommendations:
    print(recom)

In [40]:
query = "I am data scientist who loces cute dogs"
recommend(query, 5)

What is this cute puppy dog doing?#Shorts
I played weird virtual pet games
Naming My 1-Month-Old Rescue Puppies!
I asked an AI for video ideas, and they were actually good
my valentine :)


In [42]:
# Trying the function
bio = "I am from Milan and I am interested in soccer"
recommend(bio, 5)

FUT Champs but if you have a fun team I give you a free win :)
GOLDBRIDGE! AC Milan 0-1 Manchester United Match Reaction
CRIMINAL FOSSIL FOOTBALL, A DISGRACE, OUT OF EUROPE IM LIVID!🤬🤬Dinamo (3) vs Spurs (0) EXPRESSIONS
GOLDBRIDGE! Manchester United 1-1 AC Milan Match Reaction
Barcelona to change CLUB NAME and LOGO for Lionel Messi!


In [46]:
bio = "I liked the video We Put Our ENTIRE Team in ONE Game of Fortnite, the guys from fortnite"
recommend(bio, 5)

We Put Our ENTIRE Team in ONE Game of Fortnite...
the guys from fortnite
Surprising FaZe Jarvis With His Dream Fortnite Girls
25 minutes of Rainbow Six Siege VIBING
MY FORTNITE SKIN REVEAL


In [ ]:
bio = "I love football and nature"
recommend(bio, 15)

We Are Rangers | We Are Champions
My Dreams
Man City ARE RUINING Football! Goldbridge Reacts
CRIMINAL FOSSIL FOOTBALL, A DISGRACE, OUT OF EUROPE IM LIVID!🤬🤬Dinamo (3) vs Spurs (0) EXPRESSIONS
Video Games In Real Life
I HAVE RETURNED STRONGER THAN EVER
Inside Training: Shooting practice, five-a-side & head tennis | Mane vs Thiago, Alisson scores!
A Day in My Life ❤️
TXT Reacts to Fashion Trends, Fast Food & Movies | In or Out | Esquire
VAN NEISTAT: The Spirited Man
Luna & Venza. Leopard & Rottweiler. Cats vs dogs
Man Utd Appoint Director of Football! Man Utd News Now
Turning the garden into a trampoline park.
7 DAYS IN IRAQ... My Unbelievable Trip
I quit.


In [ ]:
bio = "married couples"
recommend(bio, 15)

WHO IS THE BEST COUPLE?!
Wedding Gift Had Them SPEECHLESS!!
New House Together?!
KSI & Callux Reveal The TRUTH about their Girlfriends
We Broke Up
I Might Be Pregnant For The Wedding!!
Meghan, Duchess of Sussex, opens up about her family
Wants and Needs
TELLING MY HUSBAND I'M PREGNANT
my valentine :)
WE'RE HAVING A BABY
We're Having A Baby
help
Pregnant!
Royal Announcement! | The Royal Wedding | Descendants


In [ ]:
history_of_watched_videos = """I Might Be Pregnant For The Wedding!!
my valentine :)
We Broke Up
"""
recommend(history_of_watched_videos, 15)

I Might Be Pregnant For The Wedding!!
my valentine :)
Pregnant!
We're Having A Baby
WE'RE HAVING A BABY
MOTHERS DAY SPECIAL.. PREGNANCY ANNOUNCEMENT!
TELLING MY HUSBAND I'M PREGNANT
We Broke Up
WE'RE HAVING A BABY!!
Valentines Day PRANKS on my Friends!!
We're Having A Baby!
FINDING OUT I'M PREGNANT (EMOTIONAL REACTION)
FINDING OUT MY SISTER IS PREGNANT!! *emotional*
Finding out I’m PREGNANT! | Brittany Xavier
SURPRISING MY HUSBAND AND CHILDREN WITH A POSITIVE PREGNANCY TEST! VERY EMOTIONAL!!


In [ ]:
history_of_watched_videos = """Milan 0-3 Inter | Inter Go Four Points Clear with HUGE Derby Win! | Serie A TIM
Lazio v Bayern Munich (1-4) | Youngster Musiala breaks English record | Champions League Highlights"""
recommend(history_of_watched_videos, 15)

Milan 0-3 Inter | Inter Go Four Points Clear with HUGE Derby Win! | Serie A TIM
Lazio v Bayern Munich (1-4) | Youngster Musiala breaks English record | Champions League Highlights
Bayern Munich vs Lazio (2-1) | Lewandowski Scores Again As Bayern Win | Champions League Highlights
Atalanta v Real Madrid (0-1) | Mendy stunner sinks 10-man Italians | Champions League Highlights
Real Madrid v Atalanta (3-1) | Benzema Stars In Comfortable Victory | Champions League Highlights
Juventus 3-0 Crotone | Ronaldo Hits a Brace as Juve Too Strong for Crotone! | Serie A TIM
AC Milan vs Manchester United (0-1) | Paul Pogba Strike Puts Reds Through | Europa League Highlights
Juventus v Porto (3-2) | Visitors win on away goals after dramatic tie | Champions League Highlights
Bayern Munich vs Borussia Dortmund (4-2) | Lewandowski's hat-trick sinks BVB | Bundesliga Highlights
Cagliari 1-3 Juventus | Ronaldo's First-Half Hat-Trick Seals Comfortable Victory | Serie A TIM
Chelsea vs Atletico Madrid (2-0) | Tu

#Exercise:
# Improving the recommendations
Generate 10 recoomendations for the queries you are interested in, then from these 10 videos, sort them by the number of likes, then recommend the top 3 that have received the highest number of likes.